In [7]:
import tensorflow as tf

tf.set_random_seed(777)  # reproducibility

# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
w1 = tf.Variable(tf.truncated_normal([784, 30]))
b1 = tf.Variable(tf.truncated_normal([30]))
w2 = tf.Variable(tf.truncated_normal([30, 10]))
b2 = tf.Variable(tf.truncated_normal([10]))

In [9]:
def sigma(x):
    #  sigmoid function
    return tf.div(tf.constant(1.0),
                  tf.add(tf.constant(1.0), tf.exp(-x)))

def sigma_prime(x):
    # derivative of the sigmoid function
    return sigma(x) * (1 - sigma(x))

In [10]:
# Forward prop
l1 = tf.add(tf.matmul(X, w1), b1)
a1 = sigma(l1)
l2 = tf.add(tf.matmul(a1, w2), b2)
y_pred = sigma(l2)

# diff
assert y_pred.shape.as_list() == Y.shape.as_list()
diff = (y_pred - Y)


# Back prop (chain rule)
d_l2 = diff * sigma_prime(l2)
d_b2 = d_l2
d_w2 = tf.matmul(tf.transpose(a1), d_l2)

d_a1 = tf.matmul(d_l2, tf.transpose(w2))
d_l1 = d_a1 * sigma_prime(l1)
d_b1 = d_l1
d_w1 = tf.matmul(tf.transpose(X), d_l1)


# Updating network using gradients
learning_rate = 0.01
step = [
    tf.assign(w1, w1 - learning_rate * d_w1),
    tf.assign(b1, b1 - learning_rate *
              tf.reduce_mean(d_b1, reduction_indices=[0])),
    tf.assign(w2, w2 - learning_rate * d_w2),
    tf.assign(b2, b2 - learning_rate *
              tf.reduce_mean(d_b2, reduction_indices=[0]))
]

In [11]:
# 7. Running and testing the training process
acct_mat = tf.equal(tf.argmax(y_pred, 1), tf.argmax(Y, 1))
acct_res = tf.reduce_sum(tf.cast(acct_mat, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [12]:
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(10)
    sess.run(step, feed_dict={X: batch_xs,
                              Y: batch_ys})
    if i % 1000 == 0:
        res, df = sess.run([acct_res, diff], feed_dict={X: mnist.test.images[:1000],
                                            Y: mnist.test.labels[:1000]})
        print(res, df)

# 8. Automatic differentiation in TensorFlow
cost = diff * diff
step = tf.train.GradientDescentOptimizer(0.1).minimize(cost)

103.0 [[ 0.33568823  0.40799901  0.93965769 ..., -0.05575758  0.84752357
   0.34763709]
 [ 0.01112938  0.48931819 -0.89212388 ...,  0.93603563  0.98605728
   0.03794075]
 [ 0.03409465 -0.91442275  0.5258655  ...,  0.878133    0.9138574
   0.02464347]
 ..., 
 [-0.99549556  0.62641865  0.04425149 ...,  0.98713815  0.93059462
   0.05261417]
 [ 0.00695446  0.09235598  0.89184773 ...,  0.76719421 -0.66835797
   0.00136296]
 [ 0.433902    0.00217931  0.16445139 ...,  0.24376757  0.73361051
  -0.99852651]]
286.0 [[  4.49694954e-02   3.48771811e-02   3.77113581e-01 ...,  -5.91363430e-01
    7.94646814e-02   8.59819800e-02]
 [  1.17010588e-03   4.48844284e-01  -9.83119667e-01 ...,   4.62010317e-02
    5.75730622e-01   1.25609385e-02]
 [  3.06687900e-03  -3.92348170e-01   5.07917479e-02 ...,   3.20550092e-02
    1.16602607e-01   6.06698915e-03]
 ..., 
 [ -9.99869227e-01   6.37917966e-02   3.19257425e-03 ...,   2.19034497e-02
    4.97409254e-02   7.97894597e-03]
 [  2.32395949e-04   1.59684177e-0

KeyboardInterrupt: 